In [ ]:
import sys

# Auto-install matplotlib and gradio if not available
try:
    import matplotlib.pyplot as plt
except ImportError:
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "matplotlib"])
    import matplotlib.pyplot as plt

try:
    import gradio as gr
except ImportError:
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "gradio"])
    import gradio as gr

# Helper function to safely convert input to float
def validate_float_input(value):
    try:
        value = float(value)
        return value
    except:
        return None

# Calculate transportation emissions based on unit system and inputs
def transportation(unit_system, distance, efficiency):
    if unit_system == 'imperial':
        annual_miles = distance * 52
        emissions = (annual_miles / efficiency) * 8.89 / 1000
        return emissions, f"{annual_miles:.0f} miles/year"
    else:
        annual_km = distance * 52
        liters_per_km = efficiency / 100
        emissions = annual_km * liters_per_km * 2.31 / 1000
        return emissions, f"{annual_km:.0f} km/year"

# Calculate energy emissions based on electricity usage
def energy(monthly_kwh):
    annual_kwh = monthly_kwh * 12
    return annual_kwh * 0.92e-3, f"{annual_kwh:.0f} kWh/year"

# Assign emissions value based on diet choice
def diet(diet_type):
    emission_factors = {
        "Vegan": 1.5,
        "Vegetarian": 1.7,
        "Pescatarian": 1.9,
        "Omnivore": 2.5
    }
    return emission_factors[diet_type], diet_type

# Assign emissions value based on recycling habit
def waste(recycling):
    return (0.25 if recycling == "Yes" else 0.6), recycling

# Generate emissions summary and performance rank
def summarize(emissions):
    total = sum(val[0] for val in emissions.values())
    summary = []
    for category, (value, detail) in emissions.items():
        summary.append(f"{category}: {value:.2f} metric tons CO2e ({detail})")
    summary.append(f"TOTAL: {total:.2f} metric tons CO2e")
    summary.append(f"Estimated impact: driving {total * 2500:.0f} miles or powering {total / 4.6:.1f} homes/year.")
    summary.append(f"Comparison to U.S. Average (16.5 tons): {total / 16.5 * 100:.1f}%")
    earths = total / 1.7
    summary.append(f"If everyone lived like you, we’d need {earths:.1f} Earths.")
    summary.append(performance_rank(total))
    return "\n".join(summary), total

# Generate performance rank message based on total emissions
def performance_rank(total):
    if total < 4:
        return "🌟 You’re in the top 20% of low emitters in the U.S.! Incredible effort!"
    elif total < 8:
        return "👏 You’re doing better than half the country. Keep going!"
    elif total < 16.5:
        return "💭 You're below the U.S. average. Keep reducing where you can."
    else:
        return "🚨 You are above the U.S. average. Try the suggestions to reduce your impact."

# Create a pie chart of the emissions breakdown
def plot_pie_chart(emissions):
    labels = []
    sizes = []
    for category, (value, _) in emissions.items():
        if value > 0:
            labels.append(category)
            sizes.append(value)
    fig, ax = plt.subplots(figsize=(5, 5))
    wedges, texts, autotexts = ax.pie(
        sizes,
        labels=labels,
        autopct='%1.1f%%',
        startangle=140,
        wedgeprops=dict(width=0.4),
        textprops=dict(color="black", fontsize=10)
    )
    ax.set_title("Carbon Footprint Breakdown", fontsize=14)
    ax.axis('equal')
    return fig

# Recommend tips based on high-emission categories
def recommend(emissions):
    total = sum(val[0] for val in emissions.values())
    threshold = 0.25
    suggestions = {
        "Transportation": "Consider carpooling, using public transport, or switching to an EV.",
        "Energy": "Switch to LED bulbs, unplug devices, or explore renewable energy.",
        "Diet": "Try reducing meat intake or adding plant-based meals.",
        "Waste": "Recycle regularly and reduce single-use plastics."
    }
    advice = []
    for category, (value, _) in emissions.items():
        if value / total >= threshold:
            advice.append(f"{category}: {suggestions[category]}")
    if not advice:
        advice.append("All categories are below the threshold. Keep up the good habits!")
    return "\n".join(advice)

# Modify emission values to simulate improvements
def apply_suggestions(emissions):
    if emissions["Transportation"][0] > 0:
        emissions["Transportation"] = (emissions["Transportation"][0] * 0.7, emissions["Transportation"][1])
    if emissions["Energy"][0] > 0:
        emissions["Energy"] = (emissions["Energy"][0] * 0.8, emissions["Energy"][1])
    if emissions["Diet"][1] != "Vegan":
        emissions["Diet"] = (1.5, "Improved (Vegan)")
    if emissions["Waste"][1] == "No":
        emissions["Waste"] = (0.25, "Now Recycling")
    return emissions

# Compare original and improved emissions for each category
def compare_emissions(before, after):
    comparison = []
    deltas = {}
    for key in before:
        b = before[key][0]
        a = after[key][0]
        change = b - a
        percent = (change / b * 100) if b else 0
        deltas[key] = change
        sign = "➖" if change > 0 else "➕"
        comparison.append(f"{key}: {b:.2f} ➝ {a:.2f} tons ({sign}{abs(percent):.1f}%)")
    most_changed = max(deltas.items(), key=lambda x: abs(x[1]))[0]
    return "\n".join(comparison), f"Biggest impact: {most_changed}"

# Calculate emissions and generate summary
def calculate(unit_system, distance, efficiency, monthly_kwh, diet_type, recycling):
    emissions = {
        "Transportation": transportation(unit_system, distance, efficiency),
        "Energy": energy(monthly_kwh),
        "Diet": diet(diet_type),
        "Waste": waste(recycling)
    }
    summary_text, total = summarize(emissions)
    pie_fig = plot_pie_chart(emissions)
    tips = recommend(emissions)
    return summary_text, pie_fig, tips

# Simulate improvements and compare with original emissions
def simulate_and_compare(unit_system, distance, efficiency, monthly_kwh, diet_type, recycling):
    original = {
        "Transportation": transportation(unit_system, distance, efficiency),
        "Energy": energy(monthly_kwh),
        "Diet": diet(diet_type),
        "Waste": waste(recycling)
    }
    improved = apply_suggestions(original.copy())
    summary_text, _ = summarize(improved)
    pie_fig = plot_pie_chart(improved)
    tips = recommend(improved)
    comparison_text, change_summary = compare_emissions(original, improved)
    return summary_text, pie_fig, tips, comparison_text, change_summary

# Define the Gradio interface using Tabs for step-by-step interaction
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    with gr.Tab("🌱 Start Here"):
        gr.Markdown("""
        # Welcome!
        This friendly calculator helps you estimate your annual carbon footprint.

        👉 Just walk through the next few steps to learn how your lifestyle stacks up — and how you can reduce your impact.
        """)
        gr.Markdown("Click the next tab to begin!")

    with gr.Tab("🚗 Transportation"):
        unit_system = gr.Radio(label="Unit System", choices=["imperial", "metric"], value="imperial", info="Choose miles vs. kilometers.")
        with gr.Row():
            distance = gr.Number(label="Weekly Distance Driven", value=100, scale=1)
            efficiency = gr.Number(label="Fuel Efficiency", value=25, scale=1)

    with gr.Tab("⚡ Energy"):
        monthly_kwh = gr.Number(label="Monthly Electricity Usage (kWh)", value=300)

    with gr.Tab("🥗 Diet & ♻️ Waste"):
        diet_type = gr.Radio(label="Diet Type", choices=["Vegan", "Vegetarian", "Pescatarian", "Omnivore"], value="Omnivore", scale=1)
        recycling = gr.Radio(label="Recycle Regularly?", choices=["Yes", "No"], value="Yes", scale=1)

    with gr.Tab("📊 Results"):
        gr.Markdown("## 📊 Your Carbon Footprint Summary")
        gr.Markdown("_Here's how your lifestyle choices impact the planet._")

        calc_btn = gr.Button("🔍 Calculate My Footprint")
        output_text = gr.Textbox(label="Summary", lines=12, interactive=False, show_copy_button=True)
        output_image = gr.Plot(label="Pie Chart", show_label=False)
        output_tips = gr.Textbox(label="Suggestions", lines=6, interactive=False)

        gr.Markdown("### 🧪 Try Simulating Greener Choices")
        sim_btn = gr.Button("Simulate Lifestyle Improvements")

        comparison_box = gr.Textbox(label="Before vs. After Comparison", lines=6, interactive=False)
        biggest_change = gr.Textbox(label="🏅 Your Eco Rank", lines=2, interactive=False)

        with gr.Accordion("📘 Learn More About These Numbers", open=False):
            gr.Markdown("""
            - U.S. average: **16.5 tons/year**
            - Global sustainable target: **1.7 tons/year**
            - 1 ton CO2e ≈ 2,500 miles driven
            """)

        calc_btn.click(
            fn=calculate,
            inputs=[unit_system, distance, efficiency, monthly_kwh, diet_type, recycling],
            outputs=[output_text, output_image, output_tips]
        )

        sim_btn.click(
            fn=simulate_and_compare,
            inputs=[unit_system, distance, efficiency, monthly_kwh, diet_type, recycling],
            outputs=[output_text, output_image, output_tips, comparison_box, biggest_change]
        )

        gr.Markdown("---")
        gr.Markdown("_Built with 💡 at the Hackathon | Estimated results only | Let's reduce together!_")

# Run the Gradio app
if __name__ == "__main__":
    demo.launch()
